### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

### Ensure correct package versions

In [ ]:
!pip show oci

In [ ]:
!pip install oci -U

In [ ]:
!pip show oci-cli

In [ ]:
!pip install oci-cli -U

In [ ]:
!pip install oracle-ads -U

In [ ]:
import os
import ads

In [ ]:
ads.hello()

### Set parameters for API call

In [ ]:
# input_object_storage_namespace_name=
# output_object_storage_namespace_name=
# input_bucket_name=
# output_bucket_name=input_bucket_name
# output_object_storage_prefix="Document_Analysis"
# object_name=
# compartment_id=os.environ["NB_SESSION_COMPARTMENT_OCID"]
# display_name="document_analysis_processor_job"
# language="en"
# opc_retry_token=None
# opc_request_id=None
# document_type="BANK_STATEMENT"

In [ ]:
input_object_storage_namespace_name=
output_object_storage_namespace_name=input_object_storage_namespace_name
input_bucket_name=
output_bucket_name=input_bucket_name
output_object_storage_prefix="Document_Analysis"
object_name=
compartment_id=os.environ["NB_SESSION_COMPARTMENT_OCID"]
display_name="document_analysis_processor_job"
language="en"
opc_retry_token=None
opc_request_id=None
document_type="BANK_STATEMENT"

### API call for generating document analysis

In [ ]:
# import oci

# # Refer to
# # https://docs.cloud.oracle.com/en-us/iaas/Content/API/Concepts/sdkconfig.htm#SDK_and_CLI_Configuration_File
# # for more info
# ads.set_auth(auth='resource_principal')
# rps = oci.auth.signers.get_resource_principals_signer()

# # Initialize service client with resource principal authentication
# ai_document_client = oci.ai_document.AIServiceDocumentClient(config={}, signer=rps)


# # Send the request to service, some parameters are not required, see API
# # doc for more info
# create_processor_job_response = ai_document_client.create_processor_job(
#     create_processor_job_details=oci.ai_document.models.CreateProcessorJobDetails(
#         input_location=oci.ai_document.models.ObjectStorageLocations(
#             source_type="OBJECT_STORAGE_LOCATIONS",
#             object_locations=[
#                 oci.ai_document.models.ObjectLocation(
#                     namespace_name=input_object_storage_namespace_name,
#                     bucket_name=input_bucket_name,
#                     object_name=object_name)]),
#         output_location=oci.ai_document.models.OutputLocation(
#             namespace_name=output_object_storage_namespace_name,
#             bucket_name=output_bucket_name,
#             prefix=output_object_storage_prefix),
#         compartment_id=compartment_id,
#         processor_config=oci.ai_document.models.GeneralProcessorConfig(
#             processor_type="GENERAL",
#             features=[oci.ai_document.models.DocumentTextExtractionFeature(
#                     feature_type="LANGUAGE_CLASSIFICATION",generate_searchable_pdf=True),
#                      oci.ai_document.models.DocumentTextExtractionFeature(
#                     feature_type="TABLE_EXTRACTION",generate_searchable_pdf=True),
#                      oci.ai_document.models.DocumentTextExtractionFeature(
#                     feature_type="TEXT_EXTRACTION",generate_searchable_pdf=True),
#                      oci.ai_document.models.DocumentTextExtractionFeature(
#                     feature_type="KEY_VALUE_EXTRACTION",generate_searchable_pdf=True),
#                      oci.ai_document.models.DocumentTextExtractionFeature(
#                     feature_type="DOCUMENT_CLASSIFICATION",generate_searchable_pdf=True)],
#             document_type=document_type,
#             is_zip_output_enabled=True,
#             language=language),
#         display_name=display_name),
#     opc_retry_token=opc_retry_token,
#     opc_request_id=opc_request_id)

# # Get the data from response
# print(create_processor_job_response.data)

In [ ]:
create_processor_job_id = create_processor_job_response.data.id

### Start here if document analysis has already completed and variables are no longer stored in memory

In [ ]:
# create_processor_job_id = # use create_processor_job_response.data.id from output of API call

In [ ]:
download_dir_list=!pwd
download_dir=download_dir_list[0]

In [ ]:
download_dir

### OCI-CLI call for downloading the document analysis

In [ ]:
!oci --auth resource_principal os object bulk-download -bn {output_bucket_name} --overwrite --namespace {output_object_storage_namespace_name} --prefix {output_object_storage_prefix}/{create_processor_job_id} --download-dir {download_dir}


### Show where JSON analysis is

In [ ]:
json_path_list = !ls {download_dir}/{output_object_storage_prefix}/{create_processor_job_id}/{output_object_storage_namespace_name}_{output_bucket_name}/results/*.json
json_path = json_path_list[0]

In [ ]:
json_path

### Define function
#### Add empty elements to preserve relational structure according to analysis

In [ ]:
import re

In [ ]:
# add empty elements where there are gaps before, between, and after columnIndex values in list_column_index
def add_empty_elements(list_column_index, list_text_sorted, ncols):
    
    list_text_sorted_with_empties=[]
    
    empty_value = None # placeholder value
    
    for i in range(ncols):
        if i in list_column_index:
            index = list_column_index.index(i)
            text = list_text_sorted[index]
            list_text_sorted_with_empties.append(text)
        else:
            list_text_sorted_with_empties.append(empty_value)
                
    return list_text_sorted_with_empties

### Remove CSV files

In [ ]:
!rm -rf {output_file_prefix}/{output_file_dir_name}/*.csv

### Parse JSON analysis to CSV

In [ ]:
# set parameters
output_file_prefix_list=!pwd
output_file_prefix=output_file_prefix_list[0]
output_file_dir_name="CSV_output"

In [ ]:
!mkdir -p {output_file_prefix}/{output_file_dir_name}

In [ ]:
import json
  
# Opening JSON file
f = open(json_path)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Closing file
f.close()

In [ ]:
def json_tables_parse(table, output_file, ncols, header):
    if header:
        row_type = "headerRows"
        
        # header row is empty -> True, else False
        if not table[row_type]:
            return True
    else:
        row_type = "bodyRows"
    
    count=0 # 0411
    
    for r in table[row_type]:
        row_text=[]
        row_column_index=[]

        for cell in r["cells"]:
            text = cell["text"]
                
            # Sort the cells based on columnIndex
            row_text.append(text)
            row_column_index.append(cell["columnIndex"])
            
        row_text_sorted = [x for _, x in sorted(zip(row_column_index,row_text))]
        row_column_index_sorted = sorted(row_column_index)
        
        row_text_sorted = add_empty_elements(row_column_index_sorted, row_text_sorted, ncols)

        with open(output_file, "a", encoding="utf-8") as f:
            write = csv.writer(f)
            if count==0: # 0411
                arr=[]
                for i in range(0,ncols):
                    arr.append(None)
                write.writerow(arr)
                count+=1
            write.writerow(row_text_sorted)

In [ ]:
import csv
filename_prefix="mydoc_"

for page in data["pages"]:
    page_number=data["pages"].index(page)
    
    for table in page["tables"]:
        table_number=page["tables"].index(table)
        ncols = table["columnCount"]
        
        #print(f"p_{page_number}_t_{table_number}")
        
        output_file = f"{output_file_prefix}/{output_file_dir_name}/{filename_prefix}p_{page_number}_t_{table_number}.csv"
        
        header_row_is_empty = json_tables_parse(table, output_file, ncols, True)
        json_tables_parse(table, output_file, ncols, False)

In [ ]:
conn = {
    "user_name": "",
    "password": "",
    "service_name": "",
    "wallet_location": ""
}

In [ ]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

In [ ]:
import pandas as pd

In [ ]:
directory=f"{output_file_prefix}/{output_file_dir_name}"
count=0
for f in listdir_nohidden(directory):
    count+=1
    
    # create file path
    filepath = os.path.join(directory, f)
    
    # read csv as dataframe from filepath
    df = pd.read_csv(filepath, sep=',', header=0, index_col=False,error_bad_lines=False)
    
    # create a table name from the file name
    tablename = os.path.splitext(f)[0]
    
    print(f"{count}:{filepath}:{tablename}")
    
    # push table to ADW from dataframe
    df.ads.to_sql(tablename,connection_parameters=conn,if_exists='replace')

### SQL query for dropping all tables with prefix MYDOC_P_